# CNN Fruit Classification

### Import Libraries and read images into lists

In [115]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL # image related operations
import glob # to retriive directory information and file selection
import PIL
from PIL import Image

In [82]:
train_images=glob.glob('train/*.jpg')
test_images = glob.glob('test/*.jpg')

Note lists below contain PIL.image objects. i.e. apple_images_train[0] is of PIL.Image type

In [103]:
apple_images_train=[Image.open(i).convert('RGB') for i in train_images if 'apple' in i]
orange_images_train=[Image.open(i).convert('RGB') for i in train_images if 'orange' in i]
banana_images_train=[Image.open(i).convert('RGB') for i in train_images if 'banana' in i]
mixed_images_train=[Image.open(i).convert('RGB') for i in train_images if 'mixed' in i]
apple_images_test=[Image.open(i).convert('RGB') for i in test_images if 'apple' in i]
orange_images_test=[Image.open(i).convert('RGB') for i in test_images if 'orange' in i]
banana_images_test=[Image.open(i).convert('RGB') for i in test_images if 'banana' in i]
mixed_images_test=[Image.open(i).convert('RGB') for i in test_images if 'mixed' in i]

### Preprocess

Resize and GetDatasets and two pre-processing functions. Resize will resize images with given height and weight and convert them to Numpy array. GetDatasets receives four arrays and concatenates them as a training dataset, it also creates a testing dataset. 

In [120]:
def Resize(list,height,weight):
    array = np.zeros((len(list),height,weight,3))
    for i in range(len(list)):
        temp = list[i].resize((height,weight))
        array[i] = np.asarray(temp)
    return array

In [121]:
def GetDatasets(array0,array1,array2,array3):
    X = np.concatenate((array0,array1,array2,array3),axis = 0)
    y_label_encoding = np.concatenate((np.zeros(array0.shape[0]),np.ones(array1.shape[0]),2 * np.ones(array2.shape[0]),3 * np.ones(array3.shape[0])),axis = 0)
    y_onehot = tf.keras.utils.to_categorical(y_label_encoding, 4)
    return (X,y_onehot)

In [122]:
apple = Resize(apple_images_train,100,100)
banana = Resize(banana_images_train,100,100)
orange = Resize(orange_images_train,100,100)
mixed = Resize(mixed_images_train,100,100)

In [123]:
apple_test = Resize(apple_images_test,100,100)
banana_test = Resize(orange_images_test,100,100)
orange_test = Resize(banana_images_test,100,100)
mixed_test = Resize(mixed_images_test,100,100)

In [124]:
X_train,y_train = GetDatasets(apple,banana,orange,mixed)

In [125]:
X_test,y_test = GetDatasets(apple_test,banana,orange,mixed)

### Model

In [95]:
def run_cnn(X_train, y_train, X_test, y_test):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), 
        activation='relu', input_shape=(100, 100, 3)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), 
        activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.summary()

    model.fit(X_train, y_train, 
        batch_size=64, epochs=10, verbose=1,
        validation_data=(X_test, y_test))
        

    score = model.evaluate(X_test, y_test)
    print("score =", score)

In [96]:
model = run_cnn(X_train, y_train, X_test, y_test)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               9437312   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)              

In [ ]:
model = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)# --->outputlayer
])